# Prepare buffered reaches for landsat temperature extraction

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path
import rasterio as rio

In [2]:
shapefile_io_dir = Path('/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/GIS/shapefiles/flowlines_to_reaches')
# shapefile_io_dir = Path('/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/GIS/shapefiles/flowlines_to_reaches')

koppen = Path(
    "/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/GIS/raw/Beck_KG_V1/Beck_KG_V1_present_0p083.tif"
)

In [3]:
filteredReachLines = gpd.read_file(shapefile_io_dir / 'filteredReachLines.shp')

# replace all null values in the Width* with 30
filteredReachLines_ = filteredReachLines.copy()
filteredReachLines_.fillna({'WidthMax': 30, 'WidthMean': 30}, inplace=True)

In [4]:
# add unique reach_id to bufferedReaches
countList = []
unique_ids = []
for row, value in filteredReachLines.iterrows():
    countList.append(value["GNIS_Name"])
    unique_ids.append('_'.join(value["GNIS_Name"].split()+[str(countList.count(value["GNIS_Name"]))]))

# bufferedReaches["reach_id"] = unique_ids

# add reach_id to filteredReachLines
filteredReachLines["reach_id"] = unique_ids

In [5]:
bufferedReaches = filteredReachLines.copy()
# bufferedReaches['geometry'] = bufferedReaches.geometry.buffer(filteredReachLines['WIDTH95']/2 + 120, resolution=5)
bufferedReaches["geometry"] = bufferedReaches.geometry.buffer(
    filteredReachLines_["WidthMean"] / 2 + 120,
    # filteredReachLines_["WidthMax"] / 2 + 120,
    resolution=5,
    # cap_style=3
)

In [6]:
koppen_raster = rio.open(koppen)
reprojected_reaches = bufferedReaches.to_crs(koppen_raster.crs)

In [7]:
reprojected_reaches.head()

,GNIS_ID,GNIS_Name,Basin,uniqueID,WidthMin,WidthMean,WidthMax,geometry,reach_id
0,01527909,Wenatchee River,Columbia River Basin,0,NaN,NaN,NaN,"POLYGON ((-120.72772 47.80910, -120.72771 47.8...",Wenatchee_River_1
1,01527909,Wenatchee River,Columbia River Basin,1,NaN,NaN,NaN,"POLYGON ((-120.66311 47.78491, -120.66323 47.7...",Wenatchee_River_2
2,01527909,Wenatchee River,Columbia River Basin,2,NaN,NaN,NaN,"POLYGON ((-120.66674 47.75771, -120.66709 47.7...",Wenatchee_River_3
3,01527909,Wenatchee River,Columbia River Basin,3,NaN,NaN,NaN,"POLYGON ((-120.71482 47.70213, -120.71495 47.7...",Wenatchee_River_4
4,01527909,Wenatchee River,Columbia River Basin,4,NaN,NaN,NaN,"POLYGON ((-120.73500 47.67660, -120.73494 47.6...",Wenatchee_River_5


In [8]:
# Extract Köppen class for each reach based on centroid and add to reaches_gdf
koppen_class = []
for i in range(len(reprojected_reaches)):
    row = reprojected_reaches.iloc[i]
    x = row.geometry.centroid.x
    y = row.geometry.centroid.y
    row, col = koppen_raster.index(x, y)
    koppen_class.append(koppen_raster
                        .read(1)[row, col])
    
bufferedReaches['koppen'] = koppen_class
filteredReachLines['koppen'] = koppen_class

In [9]:
bufferedReaches.to_file(shapefile_io_dir / 'bufferedReaches.shp')
filteredReachLines.to_file(shapefile_io_dir / 'filteredReachLines.shp')